# Dataset Preparation Tutorial

Welcome to the dataset preparation tutorial! In this notebook, we will download the toy data set for the tutorial and prepare the necessary tables used for later analysis. Here are the steps we will review:

- Verify prerequisites
- Create a new directory for your project
- Download data
- Set up project configuration files
- Build the proxy table
- Run regional annotation ETL

Please note that, for the remainder of the tutorial, we assume that you are on the LUNA servers. The following steps will not execute properly if this is not the case. 

## Verify Prerequisites

To check that that you have the software prerequisites, make sure you can run the following without errors:

In [1]:
!python3 --version
!java --version
%env JAVA_HOME=/usr
!echo $PYSPARK_PYTHON
!echo $PYSPARK_DRIVER_PYTHON
!echo $SPARK_HOME
!echo $JAVA_HOME
!echo $LUNA_HOME
!which jupyter
!pip list | grep luna
import luna
luna.__path__

Python 3.6.9
openjdk 11.0.11 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
env: JAVA_HOME=/usr
/usr/bin/python3
/usr/bin/python3

/usr
/home/laluna/luna
/home/laluna/.local/bin/jupyter
pyluna              0.0.1
pyluna-common       0.0.1
pyluna-pathology    0.0.1


_NamespacePath(['/home/laluna/luna', '/home/laluna/.local/lib/python3.6/site-packages/luna'])

If you see the outputs as shown in the [Luna User Guide](https://pyluna.readthedocs.io/en/latest/user.html), congratulations! It is as simple as that. You are ready to start making the project workspace and preparing the data!

## Create a new directory for your project



Next, we will create a project workspace for your configurations, data, models, and outputs to go for this tutorial.

To do so, we first need to create a file called *manifest.yaml* and populate its contents with those of the template configuration file.

In [2]:
%%bash

touch manifest.yaml
cp -v configs/manifest.yaml luna/manifest.yaml
cat luna/manifest.yaml

'configs/manifest.yaml' -> 'luna/manifest.yaml'
# project manifest template

# MIND project id
PROJECT: PRO_12-123

# IRB
IRB:

# project title
TITLE: pathology-tutorial

# project description
DESCRIPTION: End-to-end pathology analysis tutorial

DATA_MODALITIES: pathology

ROOT_PATH: /home/laluna/


Then configure logging for Luna. 

In [3]:
%%bash

cp configs/logging.cfg luna/conf/logging.cfg
cat luna/conf/logging.cfg

CENTRAL_LOGGING: False
MONGO_HOST: localhost
MONGO_PORT: 27017
# DEBUG, INFO, WARNING, ERROR
LOG_LEVEL: INFO

Next, we will create a new project workspace for this tutorial using the Luna CLI and the project manifest file. 

In [4]:
!cd luna && python3 -m src.luna.project.generate --manifest_file manifest.yaml
!ls -l PRO_12-123

2021-10-08 19:15:46,760 - INFO - root - FYI: Initalized logger, log file at: data-processing.log with handlers: [<StreamHandler <stderr> (INFO)>, <RotatingFileHandler /home/laluna/luna/data-processing.log (INFO)>]
2021-10-08 19:15:46,761 - INFO - luna.common.config - loading config file manifest.yaml
2021-10-08 19:15:46,775 - INFO - root - config files copied to /home/laluna/PRO_12-123
2021-10-08 19:15:46,776 - INFO - root - Code block 'generate project folder' took: 0.014451608061790466s
total 4
-rw-r--r-- 1 laluna laluna 251 Oct  8 19:15 manifest.yaml


You should now see a new directory called *PRO_12-123* with the manifest file in it. This will be your project workspace!

## Download data

The data that you will be using for this tutorial is a set of 5 whole slide images of ovarian cancer H&E slides, available in the svs file format. Whole slide imaging refers to the scanning of conventional glass slides for research purposes; in this case, these are slides that oncologists have used to inspecting cancer samples! We will download these images from Synapse, a data warehouse used for digital research. 

We will now make a folder for your data and the toy data set in this new project workspace.

In [5]:
%%bash

mkdir -p PRO_12-123/data/toy_data_set
ls -l PRO_12-123

total 4
drwxr-xr-x 3 laluna laluna  26 Oct  8 19:15 data
-rw-r--r-- 1 laluna laluna 251 Oct  8 19:15 manifest.yaml


You can find the pathology slides for your toy data set on Synapse. First, you must navigate to the Synapse website (https://www.synapse.org/) and create an account if you do not already have one. Once your account is created, open the site, search for the project ID (syn25946167) in the righthand corner, click the "Files" tab, and download the tar.gz file as a file (not as a package). This process may take a while, as you will be downloading a little under 5 GB of data onto your machine. Once downloaded, expand the tar file, and then relocate the five svs files into the host data directory that is volume mounted into the container. Then copy the data into the toy_data_set sub-directory.

In [6]:
%%bash

cp data/*.svs PRO_12-123/data/toy_data_set
ls -l PRO_12-123/data/toy_data_set

total 4695140
-rw-r--r-- 1 laluna laluna 1322921471 Oct  8 19:16 2551028.svs
-rw-r--r-- 1 laluna laluna  584611357 Oct  8 19:16 2551129.svs
-rw-r--r-- 1 laluna laluna  966069709 Oct  8 19:16 2551389.svs
-rw-r--r-- 1 laluna laluna  520642043 Oct  8 19:16 2551531.svs
-rw-r--r-- 1 laluna laluna 1413574341 Oct  8 19:16 2551571.svs


## Set up project configuration files

Next, you must set up your configuration files.

In your project workspace, make a new directory called *my_conf* and copy the contents of the *configs/* file into it. These configs have been prebuilt for this tutorial. Each config applies to a particular step in the workflow. 

In [7]:
%%bash
mkdir -p PRO_12-123/my_conf
cp -R configs/* PRO_12-123/my_conf/
ls -l PRO_12-123/my_conf

total 56
-rwxr-xr-x 1 laluna laluna  641 Oct  8 19:16 app_config.yaml
-rw-r--r-- 1 laluna laluna  368 Oct  8 19:16 collect_tiles.yaml
drwxr-xr-x 3 laluna laluna 4096 Oct  8 19:16 dsa_configs
-rw-r--r-- 1 laluna laluna 1036 Oct  8 19:16 generate_tiles.yaml
-rw-r--r-- 1 laluna laluna  686 Oct  8 19:16 generate_tiles_all_tissues.yaml
-rw-r--r-- 1 laluna laluna  641 Oct  8 19:16 infer_tiles.yaml
-rw-r--r-- 1 laluna laluna  408 Oct  8 19:16 load_slides.yaml
-rw-r--r-- 1 laluna laluna  108 Oct  8 19:16 logging.cfg
-rw-r--r-- 1 laluna laluna  251 Oct  8 19:16 manifest.yaml
-rwxr-xr-x 1 laluna laluna  271 Oct  8 19:16 project_slides.csv
-rwxr-xr-x 1 laluna laluna 2867 Oct  8 19:16 regional_annotation_config.yaml
drwxr-xr-x 4 laluna laluna 4096 Oct  8 19:16 snakemake-configs
-rw-r--r-- 1 laluna laluna  873 Oct  8 19:16 visualize_tiles.yaml
-rwxr-xr-x 1 laluna laluna 2705 Oct  8 19:16 wsi_config.yaml


Note: while you do not have to change the contents of *my_conf/app_confi.yaml*, you must fill out a few personal fields in *my_conf/data_config.yaml* and *regional_annotation_config.yaml*, namely: REQUESTOR, REQUESTOR_EMAIL, and DATE. Please take a moment to do so now, manually.

## Build the proxy table

Now, we will run the Whole Slide Image (WSI) ETL to database the slides and build a proxy table. For reference, ETL stands for extract-transform-load; it is the method that often involves cleaning data, transforming data types, and loading data into different systems. Additionally, a proxy table is a local table that points to a remote of local file or object on the file system or in an object store. The table that we will create will point to data in the project workspace.

In [9]:
%%bash
python3 -m luna.pathology.proxy_table.generate \
        -d PRO_12-123/my_conf/wsi_config.yaml \
        -a PRO_12-123/my_conf/app_config.yaml \
        -p delta


:: loading settings :: url = jar:file:/home/laluna/.local/lib/python3.6/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- wsi_record_uuid: string (nullable = true)
 |-- slide_id: string (nullable = true)
 |-- metadata: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+--------------------+--------------------+----------+--------------------+--------+--------------------+
|                path|    modificationTime|    length|     wsi_record_uuid|slide_id|            metadata|
+--------------------+--------------------+----------+--------------------+--------+--------------------+
|file:/home/laluna...|2021-10-08 19:16:...|1413574341|WSI-03662b6be585f...| 2551571|{aperio_User -> d...|
|file:/home/laluna...|2021-10-08 19:16:...|1322921471|WSI-1ba07f58166fc...| 2551028|{aperio

2021-10-08 19:17:56,881 - INFO - root - FYI: Initalized logger, log file at: data-processing.log with handlers: [<StreamHandler <stderr> (INFO)>, <RotatingFileHandler /home/laluna/data-processing.log (INFO)>]
2021-10-08 19:17:56,881 - INFO - root - data_ingestions_template: PRO_12-123/my_conf/wsi_config.yaml
2021-10-08 19:17:56,881 - INFO - root - config_file: PRO_12-123/my_conf/app_config.yaml
2021-10-08 19:17:56,881 - INFO - root - processes: ['delta']
2021-10-08 19:17:56,881 - INFO - luna.common.config - loading config file PRO_12-123/my_conf/app_config.yaml
2021-10-08 19:17:56,884 - INFO - luna.common.config - loading config file PRO_12-123/my_conf/wsi_config.yaml
2021-10-08 19:17:56,888 - INFO - luna.common.config - validating config PRO_12-123/my_conf/wsi_config.yaml against schema /home/laluna/.local/lib/python3.6/site-packages/luna/pathology/proxy_table/data_ingestion_template_schema.yml for DATA_CFG
2021-10-08 19:17:56,934 - INFO - root - config files copied to /home/laluna/PR

This step may take a while. At the end, your proxy table should be generated!

Before we view the table, we must first update it to associate patient ID's with the slides. This is necessary for correctly training and validating the machine learning model in the coming notebooks. Once the slides are divided into "tiles" in the next notebook, the tiles are split between the training and validation sets for the ML model. If the tiles do not have patient ID's associated with them, then it is possible for tiles from one individual to appear in both the training and validation of the model; this would cause researchers to have an exaggerated interpretation of the model's accuracy, since we would essentially be validating the model on information that is too near to what it has already seen. 

Note that we will not be using patient IDs associated with MSK. Instead, we will be using spoof IDs that will suffice for this tutorial. When running this workflow with real data, make sure to include the IDs safely and securely. Run the following block of code to add a 'patient_id' column to the table and store it using Spark.

In [10]:
from pyspark.sql import SparkSession

# setup spark session
spark = SparkSession.builder \
        .appName("test") \
        .master('local[*]') \
        .config("spark.driver.host", "127.0.0.1") \
        .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.HDFSLogStore") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.hadoop.dfs.client.use.datanode.hostname", "true") \
        .config("spark.driver.memory", "6g") \
        .config("spark.executor.memory", "6g") \
        .getOrCreate()

print(spark)

# read WSI delta table
wsi_table = spark.read.format("delta") .load("file:////home/laluna/PRO_12-123/tables/WSI_toy_data_set").toPandas()

# insert spoof patient ids
patient_id=[1,2,3,4,5]
wsi_table['patient_id']=patient_id

wsi_table

# convert back to a spark table (update table)
x = spark.createDataFrame(wsi_table)
x.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("file:////home/laluna/PRO_12-123/tables/WSI_toy_data_set")

Next, we may view the WSI table! This table should have the metadata associated with the WSI slides that you just collected, including the patient IDs. 

In [11]:
# read WSI delta table
wsi_table = spark.read.format("delta") \
            .load("file:////home/laluna/PRO_12-123/tables/WSI_toy_data_set").toPandas()

# view table
wsi_table


,path,modificationTime,length,wsi_record_uuid,slide_id,metadata,patient_id
0,file:/home/laluna/PRO_12-123/data/toy_data_set...,2021-10-08 19:16:17.921,1413574341,WSI-03662b6be585f8bdb1a16a175a7cfda07c4057afe5...,2551571,"{'aperio_StripeWidth': '2032', 'aperio_User': ...",1
1,file:/home/laluna/PRO_12-123/data/toy_data_set...,2021-10-08 19:16:04.072,584611357,WSI-93ccfd50a210d0b8c7589352be9036ef5abf6b4f81...,2551129,"{'aperio_StripeWidth': '2032', 'aperio_User': ...",2
2,file:/home/laluna/PRO_12-123/data/toy_data_set...,2021-10-08 19:16:09.592,520642043,WSI-12677b7d98691d1eef8043727f27878eb9fda14b65...,2551531,"{'aperio_StripeWidth': '2032', 'aperio_User': ...",3
3,file:/home/laluna/PRO_12-123/data/toy_data_set...,2021-10-08 19:16:01.933,1322921471,WSI-1ba07f58166fc2073c854dd9b00a11eaca2203ff20...,2551028,"{'aperio_Left': '12.423057', 'aperio_StripeWid...",4
4,file:/home/laluna/PRO_12-123/data/toy_data_set...,2021-10-08 19:16:07.672,966069709,WSI-f3890775a7f36c982aae28ac58de43b1852652fc20...,2551389,"{'aperio_Left': '23.100784', 'aperio_StripeWid...",5


If the table is depicted above, congratulations, you  have successfully run the Whole Slide Image (WSI) ETL to database the slides!

## Run the regional annotation ETL

The whole slide images that you downloaded are images of ovarian cancer, but not every pixel on each slide is a tumor. In fact, the images show tumor cells, normal ovarian cells, necrosis (dead cells), fibrosis (scarred cells), and more. Pathologists at Memorial Sloan Kettering examined each slide and denoted these different features by hand, providing us with regional annotations. You may think of regional annotations as scientific highlighter marks over the different regions of the image.

What actually happens when the regional annotation ETL is run? First, annotation bitmaps are downloaded from SlideViewer, a repository which stores WSI images and their annotation data. These bitmaps are converted into numpy arrays, which are then converted into GeoJSON files and organized in the proxy table. The GeoJSON files store the annotation regions marked by pathologists as polygons, which makes the data simpler to store and analyze. Once the annotation files are loaded into QuPath- a software used for digital pathology- later in the pipeline, this data format becomes incredibly useful and easy to work with.

To run the regional annotation ETL, try:

In [13]:
%%bash

python3 -m luna.pathology.refined_table.regional_annotation.dask_generate \
        -d PRO_12-123/my_conf/regional_annotation_config.yaml \
        -a PRO_12-123/my_conf/app_config.yaml


Building contours for label 1
num_pixels with label 16475072
num_contours 5
[-1, -1, -1, 2, 2]
Building contours for label 2
num_pixels with label 3750146
num_contours 3
[-1, 0, 0]
Building contours for label 3
num_pixels with label 115784442
num_contours 11
[-1, -1, 0, 1, 1, 1, -1, 1, -1, -1, 9]
No label 4 found
Building contours for label 5
num_pixels with label 62971884
num_contours 27
[-1, 0, -1, -1, 0, -1, -1, 6, 0, -1, -1, 3, 3, 0, 0, 0, -1, 0, 0, 16, 16, 16, 16, 16, 16, 16, -1]
No label 6 found
No label 7 found
No label 8 found
No label 9 found
No label 10 found
No label 11 found
No label 12 found
No label 13 found
No label 14 found
No label 15 found
No label 1 found
Building contours for label 2
num_pixels with label 344474790
num_contours 2
[-1, 0]
No label 3 found
Building contours for label 4
num_pixels with label 62336170
num_contours 3
[-1, 0, 0]
No label 5 found
No label 6 found
Building contours for label 7
num_pixels with label 2720232
num_contours 2
[-1, -1]
No label 8

2021-10-08 19:21:35,867 - INFO - root - FYI: Initalized logger, log file at: data-processing.log with handlers: [<StreamHandler <stderr> (INFO)>, <RotatingFileHandler /home/laluna/data-processing.log (INFO)>]
2021-10-08 19:21:35,867 - INFO - luna.common.config - loading config file PRO_12-123/my_conf/regional_annotation_config.yaml
2021-10-08 19:21:35,875 - INFO - luna.common.config - loading config file PRO_12-123/my_conf/app_config.yaml
2021-10-08 19:21:35,878 - INFO - root - data template: PRO_12-123/my_conf/regional_annotation_config.yaml
2021-10-08 19:21:35,878 - INFO - root - config_file: PRO_12-123/my_conf/app_config.yaml
2021-10-08 19:21:35,904 - INFO - root - config files copied to /home/laluna/PRO_12-123/configs/REGIONAL_METADATA_RESULTS
2021-10-08 19:21:38,849 - INFO - root - FYI: Initalized logger, log file at: data-processing.log with handlers: [<StreamHandler <stderr> (INFO)>, <RotatingFileHandler /home/laluna/data-processing.log (INFO)>]
2021-10-08 19:21:38,930 - INFO - 

To check that the regional annotation ETL was correctly run, after the Jupyter cell finishes, you may load the regional annotations table! This table contains the metadata saved from running the ETL. It includes paths to the bitmap files, numpy files, and geoJSON files that were mentioned before. To load the table, run the following code cell: 

In [14]:
from pyarrow.parquet import read_table

regional_annotation_table = read_table("/home/laluna/PRO_12-123/tables/REGIONAL_METADATA_RESULTS",
                                      filters = [('user', '!=', f'CONCAT')]).to_pandas()
regional_annotation_table


,sv_project_id,slideviewer_path,slide_id,user,bmp_filepath,npy_filepath,geojson_path,date,labelset
0,134,2019;HobS19-409411851898;2551028.svs,2551028,ellensol,./regional_bmps/2019_HobS19-409411851898_25510...,./regional_npys/2019_HobS19-409411851898_25510...,./slides/2551028/ellensol/RegionalAnnotationJS...,2021-10-08 19:21:39.209640,DEFAULT_LABELS
1,134,2019;HobS19-409411851898;2551028.svs,2551028,ellensol,./regional_bmps/2019_HobS19-409411851898_25510...,./regional_npys/2019_HobS19-409411851898_25510...,./slides/2551028/ellensol/RegionalAnnotationJS...,2021-10-08 19:21:39.209640,PIXEL_CLASSIFIER_LABELS
2,134,2019;HobS19-409411851898;2551028.svs,2551028,ellensol,./regional_bmps/2019_HobS19-409411851898_25510...,./regional_npys/2019_HobS19-409411851898_25510...,./slides/2551028/ellensol/RegionalAnnotationJS...,2021-10-08 19:21:39.209640,OBJECT_CLASSIFIER_LABELS
3,134,2019;HobS19-409411851898;2551028.svs,2551028,ellensol,./regional_bmps/2019_HobS19-409411851898_25510...,./regional_npys/2019_HobS19-409411851898_25510...,./slides/2551028/ellensol/RegionalAnnotationJS...,2021-10-08 19:21:39.209640,SIMPLIFIED_PIXEL_CLASSIFIER_LABELS
4,134,2019;HobS19-159147602774;2551129.svs,2551129,ellensol,./regional_bmps/2019_HobS19-159147602774_25511...,./regional_npys/2019_HobS19-159147602774_25511...,./slides/2551129/ellensol/RegionalAnnotationJS...,2021-10-08 19:21:39.210080,DEFAULT_LABELS
5,134,2019;HobS19-159147602774;2551129.svs,2551129,ellensol,./regional_bmps/2019_HobS19-159147602774_25511...,./regional_npys/2019_HobS19-159147602774_25511...,./slides/2551129/ellensol/RegionalAnnotationJS...,2021-10-08 19:21:39.210080,PIXEL_CLASSIFIER_LABELS
6,134,2019;HobS19-159147602774;2551129.svs,2551129,ellensol,./regional_bmps/2019_HobS19-159147602774_25511...,./regional_npys/2019_HobS19-159147602774_25511...,./slides/2551129/ellensol/RegionalAnnotationJS...,2021-10-08 19:21:39.210080,OBJECT_CLASSIFIER_LABELS
7,134,2019;HobS19-159147602774;2551129.svs,2551129,ellensol,./regional_bmps/2019_HobS19-159147602774_25511...,./regional_npys/2019_HobS19-159147602774_25511...,./slides/2551129/ellensol/RegionalAnnotationJS...,2021-10-08 19:21:39.210080,SIMPLIFIED_PIXEL_CLASSIFIER_LABELS
8,134,2019;HobS19-475053909405;2551389.svs,2551389,soslowr,./regional_bmps/2019_HobS19-475053909405_25513...,./regional_npys/2019_HobS19-475053909405_25513...,./slides/2551389/soslowr/RegionalAnnotationJSO...,2021-10-08 19:21:39.210302,DEFAULT_LABELS
9,134,2019;HobS19-475053909405;2551389.svs,2551389,soslowr,./regional_bmps/2019_HobS19-475053909405_25513...,./regional_npys/2019_HobS19-475053909405_25513...,./slides/2551389/soslowr/RegionalAnnotationJSO...,2021-10-08 19:21:39.210302,PIXEL_CLASSIFIER_LABELS


At this point, you have successfully set up your workspace, dowloaded the data, and run both the pathology and regional annotation ETLs to prepare your data. You are ready to move on to the tiling notebook!